In [ ]:
!pip install watermark

In [2]:
%load_ext watermark

In [3]:
%watermark -a "Rishit Kapoor 21BAI1046" -u -d -v -m

Author: Rishit Kapoor 21BAI1046

Last updated: 2024-02-23

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.58+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [4]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'traffic-sign-dataset-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1794080%2F2952603%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240223%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240223T071759Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D97644cf96407d50e09ca84cee41599da049cee6b48fd2ef7f9ca8a5f7aa64b325f1f2d09c55fcb64bfd097bd1fd6259a1dddd7a2f5ed710f1d3b9b12efedae5953e9942e10a8029f3c14dc89ba2800c087b40b5aac61dc74bb933ccccff6c99744a19814ecd034729a3edd3f4aeef0d772619252578a2d0ef9c0bee1bcee3dc3e0d22195b4b64377e7677ba9e454d0ab149ed8d8bd637b63d352cecec6d81ebb8cd20fdba1bd86159b6e1b0e06535e9daabc0cc01804df5694e4974e53ba03d64dc40a11ef36194bcc8c0652c2727b4e964bbbd64620e4ddfbb8288d844c44cacea34c9ef7a89898cc2dafe91e2fdb12f8d6fe3e39e46870ffdd7c988cc06e34'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 199059936 bytes downloaded
Downloaded and uncompressed: traffic-sign-dataset-classification
Data source import complete.


In [5]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.applications.resnet50 import ResNet50
from skimage.io import  imread, imshow
from skimage.transform import  resize, rescale
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [6]:
traindata_gen = ImageDataGenerator(zoom_range=0.5, shear_range=0.8, horizontal_flip=True, rescale=1/255)

In [7]:
path='/kaggle/input/traffic-sign-dataset-classification/traffic_Data/DATA'
data_train_gen =traindata_gen.flow_from_directory(
    directory=path,
    target_size=(224,224),
    batch_size=3,
    class_mode="categorical",
    color_mode='rgb',
    seed = 1234,
    shuffle = True
    )

Found 4170 images belonging to 58 classes.


In [8]:
data_train_gen.class_indices

{'0': 0,
 '1': 1,
 '10': 2,
 '11': 3,
 '12': 4,
 '13': 5,
 '14': 6,
 '15': 7,
 '16': 8,
 '17': 9,
 '18': 10,
 '19': 11,
 '2': 12,
 '20': 13,
 '21': 14,
 '22': 15,
 '23': 16,
 '24': 17,
 '25': 18,
 '26': 19,
 '27': 20,
 '28': 21,
 '29': 22,
 '3': 23,
 '30': 24,
 '31': 25,
 '32': 26,
 '33': 27,
 '34': 28,
 '35': 29,
 '36': 30,
 '37': 31,
 '38': 32,
 '39': 33,
 '4': 34,
 '40': 35,
 '41': 36,
 '42': 37,
 '43': 38,
 '44': 39,
 '45': 40,
 '46': 41,
 '47': 42,
 '48': 43,
 '49': 44,
 '5': 45,
 '50': 46,
 '51': 47,
 '52': 48,
 '53': 49,
 '54': 50,
 '55': 51,
 '56': 52,
 '57': 53,
 '6': 54,
 '7': 55,
 '8': 56,
 '9': 57}

In [9]:
all_images = []
for img_path in glob.glob("/kaggle/input/traffic-sign-dataset-classification/traffic_Data/TEST/*"):
    img = Image.open(img_path)
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_array = img_array / 255.0  # divide by 255.0 to get float values between 0 and 1 (Rescale)
    all_images.append(img_array)

# Resnet #

In [10]:
ResNet50_model = ResNet50(weights='imagenet',
                  include_top=False,
                  input_shape=(224,224,3))

94765736/94765736 [==============================] - 0s 0us/step


In [11]:
ResNet50_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [12]:
ResNet50_model.output

<KerasTensor: shape=(None, 7, 7, 2048) dtype=float32 (created by layer 'conv5_block3_out')>

In [13]:
f1=Flatten()(ResNet50_model.output)
final_layer=Dense(58,activation='Softmax')(f1)
final_layer

<KerasTensor: shape=(None, 58) dtype=float32 (created by layer 'dense')>

In [14]:
model=Model(inputs=ResNet50_model.input,outputs=final_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [15]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model_training = model.fit_generator(data_train_gen, epochs=10)

Epoch 1/10
1390/1390 [==============================] - 120s 69ms/step - loss: 3.8223 - accuracy: 0.2396
Epoch 2/10
1390/1390 [==============================] - 92s 66ms/step - loss: 2.1008 - accuracy: 0.4571
Epoch 3/10
1390/1390 [==============================] - 93s 67ms/step - loss: 1.4486 - accuracy: 0.6309
Epoch 4/10
1390/1390 [==============================] - 93s 67ms/step - loss: 1.1028 - accuracy: 0.7079
Epoch 5/10
1390/1390 [==============================] - 93s 67ms/step - loss: 0.8927 - accuracy: 0.7628
Epoch 6/10
1390/1390 [==============================] - 92s 66ms/step - loss: 0.7253 - accuracy: 0.8070
Epoch 7/10
1390/1390 [==============================] - 92s 66ms/step - loss: 0.6484 - accuracy: 0.8345
Epoch 8/10
1390/1390 [==============================] - 92s 66ms/step - loss: 0.5531 - accuracy: 0.8573
Epoch 9/10
1390/1390 [==============================] - 92s 66ms/step - loss: 0.4915 - accuracy: 0.8753
Epoch 10/10
1390/1390 [==============================] - 91s 66

# Inception #

In [17]:
from tensorflow.keras.applications import InceptionV3

In [18]:
InceptionV3_model = InceptionV3(weights='imagenet',
                  include_top=False,
                  input_shape=(224,224,3))

87910968/87910968 [==============================] - 0s 0us/step


In [19]:
f2=Flatten()(InceptionV3_model.output)
final_layer=Dense(58,activation='Softmax')(f2)
final_layer

<KerasTensor: shape=(None, 58) dtype=float32 (created by layer 'dense_1')>

In [20]:
model1=Model(inputs=InceptionV3_model.input,outputs=final_layer)
model1.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalization[0]

In [21]:
model1.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
model_training = model1.fit_generator(data_train_gen, epochs=12)

Epoch 1/12
1390/1390 [==============================] - 121s 70ms/step - loss: 4.4872 - accuracy: 0.0930
Epoch 2/12
1390/1390 [==============================] - 98s 70ms/step - loss: 3.9284 - accuracy: 0.1173
Epoch 3/12
1390/1390 [==============================] - 95s 68ms/step - loss: 3.7989 - accuracy: 0.1434
Epoch 4/12
1390/1390 [==============================] - 96s 69ms/step - loss: 3.2246 - accuracy: 0.2604
Epoch 5/12
1390/1390 [==============================] - 95s 68ms/step - loss: 2.5218 - accuracy: 0.3463
Epoch 6/12
1390/1390 [==============================] - 94s 67ms/step - loss: 2.0768 - accuracy: 0.4264
Epoch 7/12
1390/1390 [==============================] - 95s 69ms/step - loss: 1.6041 - accuracy: 0.5487
Epoch 8/12
1390/1390 [==============================] - 95s 68ms/step - loss: 1.3047 - accuracy: 0.6350
Epoch 9/12
1390/1390 [==============================] - 94s 68ms/step - loss: 1.0564 - accuracy: 0.6976
Epoch 10/12
1390/1390 [==============================] - 93s 67